In [8]:
import json

import cv2
import requests
import sys
import numpy as np
from numpy import dot
from numpy.linalg import norm

LIMIT_PX = 1024
LIMIT_BYTE = 1024 * 1024  # 1MB
LIMIT_BOX = 40

In [31]:
def kakao_ocr_resize(image_path: str):
    """
    ocr detect/recognize api helper
    ocr api의 제약사항이 넘어서는 이미지는 요청 이전에 전처리가 필요.

    pixel 제약사항 초과: resize
    용량 제약사항 초과  : 다른 포맷으로 압축, 이미지 분할 등의 처리 필요. (예제에서 제공하지 않음)

    :param image_path: 이미지파일 경로
    :return:
    """
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    if LIMIT_PX < height or LIMIT_PX < width:
        ratio = float(LIMIT_PX) / max(height, width)
        image = cv2.resize(image, None, fx=ratio, fy=ratio)
        height, width, _ = height, width, _ = image.shape

        # api 사용전에 이미지가 resize된 경우, recognize시 resize된 결과를 사용해야함.
        image_path = "{}_resized.jpg".format(image_path)
        cv2.imwrite(image_path, image)

        return image_path
    return None


def kakao_ocr(image_path: str, appkey: str):
    """
    OCR api request example
    :param image_path: 이미지파일 경로
    :param appkey: 카카오 앱 REST API 키
    """
    API_URL = "https://dapi.kakao.com/v2/vision/text/ocr"

    headers = {"Authorization": "KakaoAK {}".format(appkey)}

    image = cv2.imread(image_path)
    jpeg_image = cv2.imencode(".jpg", image)[1]
    data = jpeg_image.tobytes()

    return requests.post(API_URL, headers=headers, files={"image": data})
def cos_sim(A, B):
      return dot(A, B)/(norm(A)*norm(B))

def main():
    # if len(sys.argv) != 3:
    #     print("Please run with args: $ python example.py /path/to/image appkey")
    # image_path, appkey = sys.argv[1], sys.argv[2]
    image_path = "./test5.jpg"
    appkey = "f570b1150e5fc3f2e0e9db7947df3997"
    resize_impath = kakao_ocr_resize(image_path)
    if resize_impath is not None:
        image_path = resize_impath
        print("원본 대신 리사이즈된 이미지를 사용합니다.")

    output = kakao_ocr(image_path, appkey).json()
    # print("[OCR] output:\n{}\n".format(json.dumps(output, sort_keys=True, indent=2)))
    vec_list = []
    for i in range(len(output['result'])-1):
        startvec = output['result'][i]['boxes'][1]
        endvec = output['result'][i+1]['boxes'][0]
        ve_x = endvec[0] - startvec[0]
        ve_y = endvec[1] - startvec[1]
        vec = [ve_x, ve_y]
        vec_list.append(vec)
        # print(elements['recognition_words'])
    result="result : "
    for i in range(len(vec_list)-1):
        cosinesim = cos_sim(vec_list[i],vec_list[i+1])
        print("index : ",i,"cosinesim : ",cosinesim)
        if cosinesim > 0.9:
            result = result + output['result'][i]['recognition_words'][0]
        else : 
            result = result + output['result'][i]['recognition_words'][0]
            result = result + "\n"
    print(result)



if __name__ == "__main__":
    main()

index :  0 cosinesim :  0.013364541428686764
index :  1 cosinesim :  0.9982615929125657
index :  2 cosinesim :  0.9951756657713045
index :  3 cosinesim :  -0.9799129082495343
index :  4 cosinesim :  0.9888267423877021
index :  5 cosinesim :  0.9871290173302687
index :  6 cosinesim :  -0.9749454949530157
index :  7 cosinesim :  0.9999160238431318
index :  8 cosinesim :  -0.991308100261029
index :  9 cosinesim :  0.9995120760870788
index :  10 cosinesim :  0.9965457582448797
index :  11 cosinesim :  -0.9958677377434414
index :  12 cosinesim :  -0.9880484410823533
index :  13 cosinesim :  0.9998133501301957
index :  14 cosinesim :  -0.9931662159351023
index :  15 cosinesim :  -0.5706317507392856
index :  16 cosinesim :  0.6625738822030289
index :  17 cosinesim :  0.9993148337667671
index :  18 cosinesim :  -0.9999991274754069
index :  19 cosinesim :  0.9974572108642054
index :  20 cosinesim :  -0.9946163666641329
index :  21 cosinesim :  -0.9982963386239322
index :  22 cosinesim :  -0.997